# > Problem Statement
You're a marketing analyst and you've been told by the Senior Marketing Manager that recent
marketing campaigns have not been as effective as they were expected to be. You need to analyze
the data set in order to understand this problem and propose data-driven solutions. You are required
to solve the following questions to generate a report for your management

# Import Datasets

Q1. Import necessary libraries and load the dataset and display random 5 samples. Check the info of the data and write your findings.

In [269]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings 
warnings.filterwarnings('ignore')

data = pd.read_csv('../input/marketing-data/marketing_data.csv')

data.head()

In [270]:
data.info()

- The shape of table is (2240,28) , i.e 2240 rows, 28 columns
- Out of 28 columns, 23 columns are of int data type, and rest 5 are of object


> Q2. Check the following using an appropriate method and write your findings (1 point)
* i) Check how spread out or varied your data set is.
* ii) Check where the middle 50% of your data lies.
* iii) Check boundaries for the lower, middle and upper quarters of data.



In [271]:
data.describe()

In [272]:
data.describe(include=[object])

In [273]:
df_data = data.select_dtypes(include=np.number)
for i in df_data:
    sns.boxplot(df_data[i])
    plt.show();

i) 
- DOB of customers varies from year 1893 to 1996
- Number of kids at home varies from 0 to maximum of 2 kids
- Number of teens at home: 0 to max 2 again
- Last purchase of customers were made ranging from present day data was taken to 99 days from that time.
- Highest Education is Graduation
- highest income of person is 7.5K$
- Most customers are from spain



# Missing Values

> Q3) Check for any missing values in the dataset and handle them using an appropriate method. 


In [274]:
data.isna().sum()

In [275]:
#Converting "Income" column to suitable to use

data.rename(columns={' Income ' : 'Income'}, inplace=True)
data['Income']=data["Income"].str.replace("$","")
data['Income']=data["Income"].str.replace(",","")
data['Income']=data["Income"].astype(float)
data['Income']

In [276]:
data['Income'].isnull().sum()/len(data)*100

"Income" Column has 24 null values, which is just 1% of our entire dataset


In [277]:
data['Income']

In [278]:
data['Income'].describe()

In [279]:
plt.title('Income',size=15)
sns.boxplot(x='Income',data=data);

In [280]:
plt.grid()
sns.distplot(data['Income']);

Using median values to fill null values to avoid outliers.

In [281]:
data['Income']= data['Income'].fillna(data['Income'].median())
data['Income'].isna().sum()

> Q4) Check for any presence of special characters in any variables. If present, clean/replace and
change the datatype of the variable if required. 

In [282]:
data.head()

In [283]:
data.info()

Doing the following changes:
* change Dt_Customer to date datatype

Income is already changed to datatype - float

In [284]:
data['Dt_Customer'] = pd.to_datetime(data['Dt_Customer'])
data.info()

# Age of Cutomers

> Q5) The Marketing Manager wants to know the 'Age' of the customers. Extract the feature age
from the given dataset and display the statistical summary of the age?

In [285]:
from datetime import date
data['Age']  = date.today().year - data['Year_Birth']
data.head()

In [286]:
sns.histplot(x=data['Age']);

There looks like 2 people have been living for more than 120 Years 
To get better look at our graph will drop all rows whose age > 100

In [287]:
data.shape

In [288]:
data2 = data[data['Age']>100]
data2.head()

In [289]:
data = data[data['Age']<100]
data = data[data['Income']<600000]
data.shape

In [290]:
sns.histplot(x=data['Age']);

In [291]:
plt.figure(figsize=(13,10),dpi=100)
plt.title('Age Of Customers')
ax = sns.countplot(x=data['Age']);
for p in ax.patches: 
    ax.annotate('{:}'.format(p.get_height()), (p.get_x()+0.25, p.get_height()+0.01)) 
plt.show()

In [292]:
data['Age'].describe()

# Total Amount Spent on Products

> Q6) The Marketing manager wants to understand the total amount spent on various products so
that we can find what percentage of the amount is spent on which product.
1.  Find out the total amount spent by a customer.
2.  Display the Percentage of the amount spent on Wines and other products.

In [293]:
data.nunique()

In [294]:
data['Total Amount'] = data['MntWines']+data['MntFruits']+data['MntMeatProducts']+data['MntFishProducts']+data['MntSweetProducts']+data['MntGoldProds']
data.head()

In [295]:
sns.scatterplot(x=data['Age'],y=data['Total Amount']);

In [296]:
data.filter(regex='Mnt',axis=1)

In [297]:
data['PercentWine'] = (data['MntWines']/data['Total Amount'])*100

In [298]:
data['PercentOthers'] = ((data['MntFruits']+data['MntMeatProducts']+data['MntFishProducts']+data['MntSweetProducts']+data['MntGoldProds'])/data['Total Amount'])*100

In [299]:
data.filter(regex='Percent|Total|MntWines',axis=1)

# Purchases made  via different channels

> Q7) Being an Analyst understands the total number of purchases made through different channels
which can help find the percentage these channels contribute. 
1.  Find out the total purchases done by a customer through different channels.
2. Display the percentage of the store and other channels’ contribution to the total
purchases.


In [300]:
data['Total_Purchase']= data['NumWebPurchases']+data['NumCatalogPurchases']+data['NumStorePurchases']
data.head()

In [301]:
data.filter(regex='WebP|Catalog|Store|Total_Purchase',axis=1)

In [302]:
#Percentage of store purchases

data['PergStore'] = (data['NumStorePurchases']/data['Total_Purchase'])*100

#Percentage of other channel purchases

data['PergOthers'] = ((data['NumCatalogPurchases']+data['NumWebPurchases'])/data['Total_Purchase'])*100

#Display the percentages

data.filter(regex='Store|Total_P|Perg',axis=1)

# Performance of Marketing Campaigns

> Q8) The marketing manager wants to understand the performance of different marketing
campaigns. Find out which marketing campaign is most successful? Use suitable graphs for
visualization. (Hint:- use features like AcceptedCmp for campaign information)


In [303]:
data.columns

In [304]:
campaigns = pd.DataFrame(data.iloc[:,20:25].sum(), columns=['amount']).reset_index()
campaigns.head()

In [305]:
plt.figure(figsize=(10,5))

plt.title('Success of Each Campaigns')

yx = sns.barplot(x='index',y='amount',data=campaigns);
plt.xlabel('Campaigns')
plt.ylabel('Count')
for p in yx.patches: 
    yx.annotate('{:}'.format(p.get_height()), (p.get_x()+0.25, p.get_height()+0.01)) 
plt.show()

Clearly Campaign 4 was most success of all

# Revenue Generated by different products

> Q9)The marketing manager wants to understand which products are performing the best and
which are performing the least in terms of revenue. Being an analyst, analyse the data and plot
a suitable graph to display a report on revenue generated by different products. 

In [306]:
revenue = pd.DataFrame(data.iloc[:,9:15].sum(),columns=['rev_generated']).reset_index()
revenue

In [307]:
plt.figure(figsize=(11,7))
plt.title("Revenue Generated By Products")
yyx= sns.barplot(x='index',y='rev_generated',data=revenue);
plt.xlabel('Products')
plt.ylabel('Revenue Generated')
for p in yyx.patches: 
    yyx.annotate('{:}'.format(p.get_height()), (p.get_x()+0.25, p.get_height()+0.01)) 
plt.show()

The best product is wine.
The least popular products are Fruits and Sweets

# Patten b/w age and last campaing acceptance


> Q10) The team wants to understand if there’s any pattern between the age of customers and the
last campaign acceptance rate. Plot a suitable graph to visualize the distribution of the age
with respect to customers who accepted the last campaign.

In [308]:
sns.relplot(x='Age',y='Income',hue='Response',data=data);
plt.title('Age v/s Income');

In [309]:
sns.pairplot(vars=['Age','Income'],hue='Response',data=data);

Yes there is a slight pattern visible where customers between age 25-50, who have income more than 60K$ are the most recent customers 

# Country with most accepted in last campaign

> Q11)The Chief Marketing specialist wants to visually see which Country has the most number of
customers who accepted the last campaign. What is your approach?

In [310]:
plt.figure(figsize=(12,6))
cs = sns.countplot(x='Country',hue='Response',data=data)
plt.title('Customers Acceptance of last campaign response based on country');
for p in cs.patches: 
    cs.annotate('{:}'.format(p.get_height()), (p.get_x()+0.25, p.get_height()+0.01)) 
plt.show()

We can see that most customers who accepted the last campaign are from Spain

# Total amount/purchase v/s Dependents

> Q12)Analyse visually and write your inferences about the relationship between the following:-
1. Total amount spent Vs Dependents. (Dependents=['Kidhome']+['Teenhome'])
2. Total Purchases Vs Dependents.


In [311]:
data.columns

In [312]:
data['Dependents']=data['Kidhome']+data['Teenhome']
data.head()

In [313]:
plt.figure(figsize=(10,5))
sns.swarmplot(x='Dependents',y='Total Amount',data=data);
plt.title("TOTAL AMOUNT SPENT V/S DEPENDENTS");

We can see the frequency of spending more with customers who have no dependents and gradually the frequency of spending and total amount spent is falling down with increase of dependents.

In [314]:
plt.figure(figsize=(9,5))
sns.swarmplot(x='Dependents',y='Total_Purchase',data=data);
plt.title('TOTAL NUMBER OF PURCHASES V/S DEPENDENTS');

The Total purchases is quite constant with customers having 0 or 1 dependents , and with customers having 2 dependents there is slight decrease but a huge downfall with those with 3 dependents

# Correlation analysis

> Q13) Perform Correlation Analysis and write your key inferences. 

In [315]:
data.corr()

In [327]:
datacor = data[['Age','Education','Marital_Status','Dependents','Income','Total Amount','Total_Purchase','PercentWine','Recency','NumWebVisitsMonth','Response','Complain']]
datacor

In [330]:
sns.pairplot(datacor);

# Education v/s complaints

Q14) Understand the Education background of the customers who complained in the last 2 years.
State the Education background of the customers who have registered the most number of
complaints.

In [334]:
plt.figure(figsize=(10,5))
ec = sns.countplot(x='Education',hue='Complain',data=data)
for p in ec.patches: 
    ec.annotate('{:}'.format(p.get_height()), (p.get_x()+0.25, p.get_height()+0.01)) 
plt.show()

The most number of complaints were registerd by customers with education of Graduation.

# Some more relations

In [339]:
data.columns

Q15) .Use features 'Total_amount_spent', 'MntFruits', 'MntMeatProducts', 'MntFishProducts',
'MntSweetProducts' and, 'MntGoldProds' in x-axis and y-axis and plot the following plots. (2
points)

1. Plot a pairplot with hue as Response.
2. Plot a pairplot with hue as Education.
3. Plot a pairplot with hue as Marital Status and write your key observations.

In [343]:
sns.pairplot(vars=['Total Amount','MntFruits','MntMeatProducts','MntGoldProds'], hue='Response',data=data);

Obervations:
1. Customers having Total Amount more than 1500$, are major buyers of fruits,meat and gold products in the latest campaign
2. Customers having spent more than 500$ on meat products are major buyers of fruits and gold  products in the latest campaign

In [342]:
sns.pairplot(vars=['Total Amount','MntFruits','MntMeatProducts','MntGoldProds'], hue='Education',data=data);

Observations:


In [344]:
sns.pairplot(vars=['Total Amount','MntFruits','MntMeatProducts','MntGoldProds'], hue='Marital_Status',data=data);